# Query Snowflake Database - example

This notebook demonstrates the process of querying a Snowflake SQL database. Here I will demonstrate a ETL workflow. First, extract data from the database with queries. In the transform steps, I will perform SQL data manipulation using the python API for efficient table joins and data processing. Here agian I can query the data for local analysis. Finally, load the data into further analysis such as dashboards and data science projects. These later projects include machine learning pipelines utilizing tools such as XGBoost and neural networks.

### Note: passwords have been removed to prevent unexpected compute costs, but the data has been stored within Customer Snowflake\data\snowflake_pull.

Therefore this notebook does NOT run

In [42]:
import snowflake.connector # utilized to make connections to snowflake
import pandas as pd # for dataframe and data manipulations
import os # for path functions

## Example use case of SQL joins is demonstrated below

Here multiple tables are joined to demonstrate the powerful data processing tool of Snowflake and SQL.

In [57]:
# SELECT 
#     o.order_id, 
#     o.customer_id, 
#     o.order_date,
#     o.order_total, 
#     c.name, 
#     c.email, 
#     c.join_date, 
#     c.location,
#     oi.product_id,
#     oi.quantity,
#     p.product_name,
#     p.category,
#     p.price
# FROM orders o
# INNER JOIN customers c 
#     ON o.customer_id = c.customer_id 
# INNER JOIN order_items oi 
#     ON o.order_id = oi.order_id 
# INNER JOIN products p
#     ON oi.product_id = p.product_id
# LIMIT 10;

In [ ]:
# store general requirements
# pip freeze > requirements.txt

In [23]:
# Connect to Snowflake
# conn = snowflake.connector.connect(
#     user='xxx',
#     password='xxx',
#     account='xxx',          # e.g., xy12345.us-east-1
#     warehouse='xxx',
#     database='xxx',
#     schema='xxx'
# )

conn = snowflake.connector.connect(
    user=USER,
    password=PASSWORD,
    account=ACCOUNT,
    warehouse=WAREHOUSE,
    database=DATABASE,
    schema=SCHEMA
    )


cursor = conn.cursor()

# Run a simple query
cursor.execute("SELECT CURRENT_DATE;")
result = cursor.fetchone()
print("Today is:", result[0])

Today is: 2025-06-22


In [24]:
cursor.execute('SELECT CURRENT_WAREHOUSE();')
result = cursor.fetchone()
print(result)

('COMPUTE_WH',)


In [29]:
cursor.execute("SELECT table_schema, table_name FROM ECOMM_SYNTHETIC.INFORMATION_SCHEMA.TABLES WHERE table_type = 'BASE TABLE';")
result = cursor.fetchone()
result

('PUBLIC', 'CUSTOMERS')

## Finding the Tables and Preview the data

If I do not have in-depth knowledge of the tables before working with the database, the Information Schema provides information about tables and metadata.

Establishing a connection to the database via the name does not provide information about the data itself. This query provides a limited data set which can be utilized in python for further exploratory investigation.

Limiting the data to 5 rows helps minimize compute costs while maintaing key aspects of the database schema and provides insight into the data structure.

In [ ]:
# First query the schema for the table names
cursor.execute("""
    SELECT table_name
    FROM INFORMATION_SCHEMA.TABLES
    WHERE table_schema = 'PUBLIC' AND table_type = 'BASE TABLE';
""")

# create a list of tables using cursor.fetchall()
tables = [row[0] for row in cursor.fetchall()]

# loop through the tables and display the data
for table in tables:
    print(f"\nPreview of table: {table}")
    query = f'SELECT * FROM PUBLIC."{table}" LIMIT 5'
    cursor.execute(query)
    rows = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description]
    df = pd.DataFrame(rows, columns=columns)
    display(df.head())


Preview of table: CUSTOMERS


,CUSTOMER_ID,NAME,EMAIL,JOIN_DATE,LOCATION
0,1,Nicole Morris,donnahiggins@yahoo.com,2024-02-19,Lake Victoria
1,2,Amy Murphy,colin87@aguilar.com,2025-01-01,Port Brendachester
2,3,Dawn Mckay,dyermaria@hotmail.com,2023-09-19,Sheilamouth
3,4,Krystal Li,kristin98@gmail.com,2023-07-17,Lake Katie
4,5,Joshua Taylor,rdaniel@valentine.com,2024-07-15,Mooremouth



Preview of table: ORDERS


,ORDER_ID,CUSTOMER_ID,ORDER_DATE,ORDER_TOTAL
0,1,74,2024-12-02,269.97
1,2,63,2024-06-16,134.97
2,3,64,2025-05-15,209.97
3,4,24,2025-05-15,119.98
4,5,75,2025-06-12,199.97



Preview of table: PRODUCTS


,PRODUCT_ID,PRODUCT_NAME,CATEGORY,PRICE
0,1,Running Shoes,Footwear,89.99
1,2,Cross-Training Shoes,Footwear,79.99
2,3,Cleats,Footwear,59.99
3,4,Trail Running Shoes,Footwear,94.99
4,5,Indoor Court Shoes,Footwear,74.99



Preview of table: ORDER_ITEMS


,ORDER_ID,PRODUCT_ID,QUANTITY
0,1,1,3
1,2,22,3
2,3,4,1
3,3,25,1
4,3,1,1


## Run the query again and now store the data in the data\snowflake_pull directory

Remove the LIMIT clause to pull the full data set

In [49]:
# First query the schema for the table names
cursor.execute("""
    SELECT table_name
    FROM INFORMATION_SCHEMA.TABLES
    WHERE table_schema = 'PUBLIC' AND table_type = 'BASE TABLE';
""")

# create a list of tables using cursor.fetchall()
tables = [row[0] for row in cursor.fetchall()]

# loop through the tables and display the data - remove the LIMIT from previous
for table in tables:
    print(f"\nPreview of table: {table}")
    query = f'SELECT * FROM PUBLIC."{table}"'
    cursor.execute(query)
    rows = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description]
    df = pd.DataFrame(rows, columns=columns)

    # store each dataframe using the table name
    dir = r"..\data\snowflake_pull"
    file_name = f"{table}.csv"
    file_path = os.path.join(dir, file_name)
    df.to_csv(file_path, index=False)


Preview of table: CUSTOMERS

Preview of table: ORDERS

Preview of table: PRODUCTS

Preview of table: ORDER_ITEMS


## Data is now stored locally for further analysis

In [52]:
df_customers = pd.read_csv(r'..\data\snowflake_pull\CUSTOMERS.csv')

df_customers.head()

,CUSTOMER_ID,NAME,EMAIL,JOIN_DATE,LOCATION
0,1,Nicole Morris,donnahiggins@yahoo.com,2024-02-19,Lake Victoria
1,2,Amy Murphy,colin87@aguilar.com,2025-01-01,Port Brendachester
2,3,Dawn Mckay,dyermaria@hotmail.com,2023-09-19,Sheilamouth
3,4,Krystal Li,kristin98@gmail.com,2023-07-17,Lake Katie
4,5,Joshua Taylor,rdaniel@valentine.com,2024-07-15,Mooremouth


## Join data based on primary keys

In [66]:
query = """SELECT 
    o.order_id, 
    o.customer_id, 
    o.order_date,
    o.order_total, 
    c.name, 
    c.email, 
    c.join_date, 
    c.location,
    oi.product_id,
    oi.quantity,
    p.product_name,
    p.category,
    p.price
FROM orders o
INNER JOIN customers c 
    ON o.customer_id = c.customer_id 
INNER JOIN order_items oi 
    ON o.order_id = oi.order_id 
INNER JOIN products p
    ON oi.product_id = p.product_id;"""


cursor.execute(query)

## Cursor.description provides meta data for columns

In [67]:
cursor.description

[ResultMetadata(name='ORDER_ID', type_code=0, display_size=None, internal_size=None, precision=38, scale=0, is_nullable=True),
 ResultMetadata(name='CUSTOMER_ID', type_code=0, display_size=None, internal_size=None, precision=38, scale=0, is_nullable=True),
 ResultMetadata(name='ORDER_DATE', type_code=3, display_size=None, internal_size=None, precision=None, scale=None, is_nullable=True),
 ResultMetadata(name='ORDER_TOTAL', type_code=1, display_size=None, internal_size=None, precision=None, scale=None, is_nullable=True),
 ResultMetadata(name='NAME', type_code=2, display_size=None, internal_size=16777216, precision=None, scale=None, is_nullable=True),
 ResultMetadata(name='EMAIL', type_code=2, display_size=None, internal_size=16777216, precision=None, scale=None, is_nullable=True),
 ResultMetadata(name='JOIN_DATE', type_code=3, display_size=None, internal_size=None, precision=None, scale=None, is_nullable=True),
 ResultMetadata(name='LOCATION', type_code=2, display_size=None, internal_si

In [68]:
# use the rows for the data
rows = cursor.fetchall()

# use the first data value in the metadata for naming the columns
columns = [col[0] for col in cursor.description]

# convert the data into a pandas dataframe
df = pd.DataFrame(rows, columns=columns)

# display the dataframe
df.head()

,ORDER_ID,CUSTOMER_ID,ORDER_DATE,ORDER_TOTAL,NAME,EMAIL,JOIN_DATE,LOCATION,PRODUCT_ID,QUANTITY,PRODUCT_NAME,CATEGORY,PRICE
0,1,74,2024-12-02,269.97,Larry Freeman,hilllauren@marshall.com,2023-09-22,Perryberg,1,3,Running Shoes,Footwear,89.99
1,2,63,2024-06-16,134.97,Phyllis Morrow,christinephillips@yahoo.com,2023-10-07,Josephborough,22,3,Pickleball Paddle,Racquet Sports,44.99
2,3,64,2025-05-15,209.97,Scott Bentley,nathan45@gmail.com,2025-04-21,Charlesport,4,1,Trail Running Shoes,Footwear,94.99
3,3,64,2025-05-15,209.97,Scott Bentley,nathan45@gmail.com,2025-04-21,Charlesport,25,1,Squash Goggles,Racquet Sports,24.99
4,3,64,2025-05-15,209.97,Scott Bentley,nathan45@gmail.com,2025-04-21,Charlesport,1,1,Running Shoes,Footwear,89.99


## Write data for analysis

In [ ]:
# df.to_csv(r'..\data\snowflake_pull\joined_sales_data.csv')